<a href="https://colab.research.google.com/github/FabriTT/DataSets/blob/main/FemeninaDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [60]:
base_url = "https://femenina.com.bo/marcas/comfortflex/"
response = requests.get(url)

In [61]:
response

<Response [200]>

In [62]:
# Lista para almacenar los datos de los productos
dataNews = []

# Función para extraer los productos de una página
def scrape_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        news_items = soup.find('ul', class_='products hover-slider main-products layout-standard columns-4')

        if news_items:
            items = news_items.find_all('li')
            for item in items:
                # Nombre del producto
                nombre = item.find('h2', class_='woocommerce-loop-product__title')
                product_name = nombre.a.text.strip() if nombre and nombre.a else "N/A"

                # URL del producto
                enlace = item.find('div', class_='product-thumbnail').find('a', href=True)
                product_url = enlace['href'] if enlace else "N/A"

                # Imagen del producto
                imagen = item.find('img')
                image_url = imagen['src'] if imagen else "N/A"

                # Precio del producto
                precio = item.find('span', class_='woocommerce-Price-amount')
                product_price = precio.text.strip() if precio else "N/A"

                # Obtener descripción del producto
                product_description = scrape_description(product_url)

                # Guardar en la lista
                dataNews.append([product_name, product_url, image_url, product_price, product_description])

# Función para obtener la descripción de un producto
def scrape_description(product_url):
    try:
        response = requests.get(product_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            desc_container = soup.find('div', class_='woocommerce-product-details__short-description')
            if desc_container:
                description = desc_container.get_text(strip=True)
                return description
        return "No disponible"
    except Exception as e:
        return f"Error: {e}"

# Obtener todas las páginas disponibles
def get_all_pages(base_url):
    pages = [base_url]  # Lista con la primera página
    response = requests.get(base_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        pagination = soup.find('ul', class_='page-numbers')

        if pagination:
            links = pagination.find_all('a', class_='page-numbers', href=True)
            for link in links:
                page_url = link['href']
                if page_url not in pages:
                    pages.append(page_url)  # Agregar solo si no está repetida

    return pages

# Obtener todas las URLs de paginación
page_urls = get_all_pages(base_url)

# Scraping de cada página
for page_url in page_urls:
    print(f"Scrapeando: {page_url}")
    scrape_page(page_url)
    time.sleep(1)  # Pequeña pausa para no sobrecargar el servidor

# Crear DataFrame
df = pd.DataFrame(dataNews, columns=["Producto", "URL", "Imagen", "Precio", "Descripción"])

# Configurar pandas para evitar saltos de línea
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Imprimir el DataFrame
print(df.to_string(index=False, col_space=20))




Scrapeando: https://femenina.com.bo/marcas/comfortflex/
Scrapeando: https://femenina.com.bo/marcas/comfortflex/page/2/
Scrapeando: https://femenina.com.bo/marcas/comfortflex/page/3/
Scrapeando: https://femenina.com.bo/marcas/comfortflex/page/4/
                   Producto                                                                                       URL                                                                                     Imagen               Precio                                                                                                                                                                                                                                                                                        Descripción
              Chinela Slide                                       https://femenina.com.bo/shop/zapatos/chinela-slide/            https://femenina.com.bo/wp-content/uploads/2024/04/2255401-1-1.jpg-300x300.webp              Bs. 230     